In [2]:
!pip install transformers==4.30.2
import json
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
import numpy as np

# 1. Cargar Metadatos del Grafo (Tu diccionario)
print("📂 Cargando mapa de conocimientos (Metadata)...")
with open("itoe_metadata.json", "r") as f:
    metadata = json.load(f)

ent2id = metadata["ent2id"]
rel2id = metadata["rel2id"]
embedding_dim = metadata["dim"] # Corrected key from 'embedding_dim' to 'dim'

# Invertir diccionario para búsquedas futuras
id2ent = {v: k for k, v in ent2id.items()}

# 2. Configurar Modelo NER (PubMedBERT)
# Usamos el mismo que en la Fase 1 para consistencia
print("🧠 Cargando PubMedBERT para entender síntomas...")
model_name = "d4data/biomedical-ner-all"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

def extraer_nodos_consulta(texto_usuario):
    """
    Traduce lenguaje natural a IDs del Grafo de Conocimiento.
    """
    print(f"\n🔎 Analizando consulta: '{texto_usuario}'")

    # A. Extracción de entidades crudas
    resultados = ner_pipeline(texto_usuario)
    entidades_detectadas = [res['word'] for res in resultados]

    # B. Normalización y Mapeo (Entity Linking simple)
    # Buscamos la coincidencia más cercana en tu vocabulario ent2id
    nodos_validos = []

    for ent in entidades_detectadas:
        ent_lower = ent.lower().strip()

        # Búsqueda exacta
        if ent_lower in ent2id:
            nodos_validos.append(ent2id[ent_lower])
            print(f"   ✅ Entidad reconocida: '{ent_lower}' (ID: {ent2id[ent_lower]})")
        else:
            # Búsqueda difusa simple (si contiene la palabra)
            found = False
            for db_ent, db_id in ent2id.items():
                if ent_lower in db_ent and len(ent_lower) > 3: # Evitar matches cortos
                    nodos_validos.append(db_id)
                    print(f"   ⚠️ '{ent_lower}' mapeado a concepto del KG: '{db_ent}' (ID: {db_id})")
                    found = True
                    break
            if not found:
                print(f"   ❌ Concepto ignorado (No existe en el KG): '{ent_lower}'")

    return list(set(nodos_validos)) # Retornar IDs únicos

print("✅ Sistema de Extracción listo.")

  Using cached transformers-4.30.2-py3-none-any.whl.metadata (113 kB)
  Using cached tokenizers-0.13.3.tar.gz (314 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)
📂 Cargando mapa de conocimientos (Metadata)...
🧠 Cargando PubMedBERT para entender síntomas...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

Device set to use cuda:0


✅ Sistema de Extracción listo.


In [3]:
import torch.nn as nn
import pandas as pd

# 1. Reconstruir Arquitectura ItôE (Debe ser idéntica a la del entrenamiento)
class ItoE_Inference(nn.Module):
    def __init__(self, num_ent, num_rel, dim):
        super().__init__()
        self.ent_drift = nn.Embedding(num_ent, dim)
        self.ent_diff = nn.Embedding(num_ent, dim)
        self.rel_drift = nn.Embedding(num_rel, dim)
        self.rel_diff = nn.Embedding(num_rel, dim)

    def calcular_energia(self, h_idx, r_idx, t_idx):
        # Forward pass específico para inferencia
        h_mu = self.ent_drift(h_idx)
        t_mu = self.ent_drift(t_idx)
        r_mu = self.rel_drift(r_idx)

        h_sig = torch.abs(self.ent_diff(h_idx))
        t_sig = torch.abs(self.ent_diff(t_idx))
        r_sig = torch.abs(self.rel_diff(r_idx))

        # Predicción: P(Sintoma) + Relación -> P(Enfermedad)
        pred_mu = h_mu + r_mu
        pred_sig = h_sig + r_sig + 1e-6
        t_sig = t_sig + 1e-6

        # KL Divergence (Energía)
        trace = (pred_sig / t_sig).sum(dim=-1)
        diff = ((t_mu - pred_mu)**2 / t_sig).sum(dim=-1)
        log_det = (torch.log(t_sig) - torch.log(pred_sig)).sum(dim=-1)

        return 0.5 * (trace + diff + log_det)

# 2. Cargar Pesos Entrenados
print("⚙️ Inicializando Motor ItôE...")
num_ent = len(ent2id)
num_rel = len(rel2id)
dim = embedding_dim

motor_itoe = ItoE_Inference(num_ent, num_rel, dim)
try:
    motor_itoe.load_state_dict(torch.load("itoe_model.pth", map_location=torch.device('cpu')))
    motor_itoe.eval()
    print("✅ Modelo ItôE cargado en memoria.")
except Exception as e:
    print(f"❌ Error cargando modelo: {e}")

# 3. Función de Ranking Diagnóstico
def realizar_diagnostico_diferencial(sintomas_ids, top_k=5):
    if not sintomas_ids:
        return []

    print("\n🧮 Calculando probabilidades estocásticas...")

    # Identificar relación "related_to" o similar en tu KG
    # Si no sabes el nombre exacto, usamos el índice 0 o buscamos
    rel_key = "related_to" if "related_to" in rel2id else list(rel2id.keys())[0]
    r_id = rel2id[rel_key]

    # Preparar tensores
    todos_los_nodos = torch.arange(num_ent) # Candidatos (Enfermedades potenciales)
    r_tensor = torch.tensor([r_id] * num_ent)

    # Acumulador de energía para cada candidato
    # (Si tienes 3 síntomas, sumamos la energía de cada uno hacia la enfermedad candidata)
    energia_total = torch.zeros(num_ent)

    with torch.no_grad():
        for sintoma in sintomas_ids:
            h_tensor = torch.tensor([sintoma] * num_ent)
            # Calculamos E(Sintoma -> Enfermedad)
            energia = motor_itoe.calcular_energia(h_tensor, r_tensor, todos_los_nodos)
            energia_total += energia

    # Convertir Energía a Probabilidad (Softmax negativo)
    # Energía baja = Alta probabilidad
    probs = torch.softmax(-energia_total, dim=0)

    # Obtener Top K
    scores, indices = torch.topk(probs, k=top_k)

    diagnosticos = []
    for score, idx in zip(scores, indices):
        idx_val = idx.item()
        # Filtrar para que la enfermedad no sea uno de los síntomas de entrada
        if idx_val not in sintomas_ids:
            nombre = id2ent[idx_val]
            probabilidad = score.item()
            diagnosticos.append({"enfermedad": nombre, "probabilidad": probabilidad, "energia": energia_total[idx_val].item()})

    return diagnosticos

⚙️ Inicializando Motor ItôE...
✅ Modelo ItôE cargado en memoria.


In [5]:
# --- 3. REPORTE Y CONSULTA INTERACTIVA ---

def generar_reporte_medico(diagnosticos, umbral_confianza=0.05):
    print("\n" + "="*50)
    print("📋 REPORTE DE DIAGNÓSTICO DIFERENCIAL (SISTEMA ITÔE)")
    print("="*50)

    if not diagnosticos:
        print("⚠️  No se pudo generar un diagnóstico. Los síntomas no coinciden suficientemente con la base de conocimiento.")
        return

    # 1. Verificar convergencia / Criterio de confianza
    mejor_candidato = diagnosticos[0]
    confianza_lider = mejor_candidato['probabilidad']

    # Si la probabilidad es muy baja, el modelo está "adivinando" (alta entropía/difusión)
    es_convincente = confianza_lider > umbral_confianza

    # 2. Generación del Texto (Simulación de NLP estructurado)
    if es_convincente:
        print(f"\n✅ ANÁLISIS: El perfil sintomático converge hacia una patología principal.\n")
        print(f"   DIAGNÓSTICO PRINCIPAL: {mejor_candidato['enfermedad'].upper()}")
        print(f"   Confianza del Modelo:  {confianza_lider:.2%}")
        print(f"   Nivel de Energía:      {mejor_candidato['energia']:.4f} (Menor es mejor)\n")

        print("-" * 30)
        print("   Diagnósticos Diferenciales (Otras posibilidades):")
        for d in diagnosticos[1:]:
            print(f"   • {d['enfermedad']:<30} | Probabilidad: {d['probabilidad']:.2%}")
    else:
        print(f"\n⚠️  INCERTIDUMBRE ALTA: El diagnóstico no es concluyente.\n")
        print(f"   Motivo: El candidato principal ({mejor_candidato['enfermedad']}) solo alcanza un {confianza_lider:.2%} de probabilidad.")
        print("   Esto sugiere síntomas inespecíficos o una patología fuera del Grafo de Conocimiento actual.")
        print("\n   Posibilidades débiles detectadas (Revisar con cautela):")
        for d in diagnosticos:
            print(f"   • {d['enfermedad']:<30} ({d['probabilidad']:.2%})")

    print("\n" + "="*50)
    print("NOTA: Este es un sistema experimental basado en grafos probabilísticos. Consulte a un médico.")

# --- EJECUCIÓN INTERACTIVA ---

print("\n" + "🏥"*30)
print("      SISTEMA DE INTERCONSULTA MÉDICA - ItôE Engine")
print("🏥"*30 + "\n")
print("ℹ️  INSTRUCCIONES: Ingrese el cuadro clínico del paciente.")
print("    (El modelo fue entrenado con MIMIC/PubMed, por lo que funciona mejor en INGLÉS).")
print("    Ejemplo: 'Patient presents high fever, severe cough and chest pain'\n")

# Solicitud de entrada al usuario
consulta_paciente = input("📝 Ingrese los síntomas aquí: ")

# Validación básica
if not consulta_paciente or len(consulta_paciente.strip()) < 3:
    print("\n⚠️ Entrada vacía o muy corta. Usando caso de prueba predeterminado...")
    consulta_paciente = "Patient presents high fever and severe cough"

# 2. Paso A: Extracción de Nodos (NER)
ids_sintomas = extraer_nodos_consulta(consulta_paciente)

if ids_sintomas:
    # 3. Paso B: Inferencia Estocástica (Cálculo de Energía)
    # top_k=5 nos da las 5 enfermedades con menor energía (mayor probabilidad)
    ranking = realizar_diagnostico_diferencial(ids_sintomas, top_k=5)

    # 4. Paso C: Reporte NLP
    generar_reporte_medico(ranking)
else:
    print("\n❌ ERROR: No se detectaron entidades médicas conocidas en la consulta.")
    print("   Intente reformular usando términos médicos estándar (ej. 'pyrexia' en vez de 'hot').")


🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥
      SISTEMA DE INTERCONSULTA MÉDICA - ItôE Engine
🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥🏥

ℹ️  INSTRUCCIONES: Ingrese el cuadro clínico del paciente.
    (El modelo fue entrenado con MIMIC/PubMed, por lo que funciona mejor en INGLÉS).
    Ejemplo: 'Patient presents high fever, severe cough and chest pain'

📝 Ingrese los síntomas aquí: the patient has chest pain and half of his face is paralyzed

🔎 Analizando consulta: 'the patient has chest pain and half of his face is paralyzed'
   ❌ Concepto ignorado (No existe en el KG): 'chest'
   ⚠️ 'pain' mapeado a concepto del KG: 'Abdominal pain  unspecified site' (ID: 361)
   ❌ Concepto ignorado (No existe en el KG): 'half'
   ⚠️ 'face' mapeado a concepto del KG: 'Open wound of other and multiple sites of face  without mention of complication' (ID: 673)
   ❌ Concepto ignorado (No existe en el KG): 'paralyzed'

🧮 Calculando probabilidades estocásticas...

📋 REPORTE DE DIAGNÓSTICO DIFERENCIAL (SISTEMA ITÔE)

✅ ANÁLISIS: 